In [5]:
import cv2
import numpy as np

# Define color range for lemons (lemon color in HSV)
lower_yellow = np.array([20, 100, 100])
upper_yellow = np.array([30, 255, 255])

# Set a minimum area threshold to filter out smaller contours
min_area = 1000  # Adjust this value as needed

# Set additional filtering criteria for width and height
min_width = 100  # Minimum width of the bounding box
min_height = 100  # Minimum height of the bounding box

# Create a kernel for morphological operations
kernel = np.ones((5, 5), np.uint8)

# Open the webcam (change the index if needed)
cap = cv2.VideoCapture(0)

def preprocess_frame(frame):
    """
    Preprocess the input frame: Convert to HSV, apply noise reduction, and perform background subtraction.
    """
    # Convert the frame to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Apply Gaussian Blur to reduce noise
    blurred = cv2.GaussianBlur(hsv, (5, 5), 0)

    return blurred

def segment_objects(hsv_frame):
    """
    Segment the objects of interest from the background using color thresholding and morphological operations.
    """
    # Create a mask for lemon color
    mask_lemon = cv2.inRange(hsv_frame, lower_yellow, upper_yellow)

    # Apply morphological operations to clean up the mask
    mask_lemon = cv2.morphologyEx(mask_lemon, cv2.MORPH_CLOSE, kernel)
    mask_lemon = cv2.morphologyEx(mask_lemon, cv2.MORPH_OPEN, kernel)

    return mask_lemon

def extract_features_and_count(mask, original_frame):
    """
    Extract features such as contour area, bounding box dimensions, and count the objects.
    """
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours based on area, width, and height
    filtered_contours = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if cv2.contourArea(contour) > min_area and w > min_width and h > min_height:
            filtered_contours.append(contour)
            # Draw bounding box around detected lemon
            cv2.rectangle(original_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Count the number of detected lemons
    lemon_count = len(filtered_contours)

    return lemon_count

def display_results(frame, lemon_count):
    """
    Display the results, including the count of lemons and the processed frame.
    """
    # Display the count on the frame
    cv2.putText(frame, f"Lemons Detected: {lemon_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Lemon Detection', frame)

def save_results(lemon_count):
    """
    Save the counted results to a text file.
    """
    with open("lemon_count.txt", "a") as f:
        f.write(f"Lemons Detected: {lemon_count}\n")

def main():
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()

        if not ret:
            break

        # Preprocess the frame
        preprocessed_frame = preprocess_frame(frame)

        # Segment the objects
        mask = segment_objects(preprocessed_frame)

        # Extract features and count objects
        lemon_count = extract_features_and_count(mask, frame)

        # Display the results
        display_results(frame, lemon_count)

        # Save the results to a file (optional)
        save_results(lemon_count)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # When everything done, release the capture and close windows
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()